In [1]:
import requests
from bs4 import BeautifulSoup

baseurl = "https://finance.naver.com"
url = "https://finance.naver.com/item/sise.nhn?code=005930"

resp = requests.get(url)
resp

<Response [200]>

In [2]:
soup = BeautifulSoup(resp.text,'html.parser')


In [3]:
iframe_tag = soup.find('iframe',{'name':'day'})
iframe_tag

<iframe bottommargin="0" frameborder="0" height="360" marginheight="0" name="day" scrolling="no" src="/item/sise_time.naver?code=005930&amp;thistime=20220627161132" style="margin-top:20px;clear:both" title="주요 시세" topmargin="0" width="100%"></iframe>

In [4]:
iframe_url = iframe_tag.get('src')
iframe_url = baseurl+iframe_url
iframe_url

'https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220627161132'

In [5]:
resp1 = requests.get(iframe_url)
resp1.headers

{'date': 'Mon, 27 Jun 2022 17:22:53 GMT', 'set-cookie': 'JSESSIONID=39EEBE3722BA6061DDC2B7D19871425D; Path=/; HttpOnly', 'vary': 'Accept-Encoding,User-Agent', 'content-encoding': 'gzip', 'referrer-policy': 'unsafe-url', 'transfer-encoding': 'chunked', 'content-type': 'text/html;charset=EUC-KR', 'x-xss-protection': '1; mode=block', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'server': 'nfront'}

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36 Edg/103.0.1264.37'}
resp1 = requests.get(iframe_url,headers=headers)
resp1.headers




{'date': 'Mon, 27 Jun 2022 17:22:53 GMT', 'cache-control': 'no-cache', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'set-cookie': 'JSESSIONID=713341CD2BA1D41F4FB93359E7BDA615; Path=/; HttpOnly', 'content-language': 'ko', 'vary': 'Accept-Encoding,User-Agent', 'content-encoding': 'gzip', 'referrer-policy': 'unsafe-url', 'transfer-encoding': 'chunked', 'content-type': 'text/html;charset=EUC-KR', 'x-xss-protection': '1; mode=block', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'server': 'nfront'}

In [7]:
soup = BeautifulSoup(resp1.text,'html.parser')

In [8]:
columns = []
columns_tag = soup.select('tr th')
for i in columns_tag:
    columns.append(i.text)
columns

['체결시각', '체결가', '전일비', '매도', '매수', '거래량', '변동량']

In [9]:
import pandas as pd
data = pd.DataFrame(columns=columns)
data

,체결시각,체결가,전일비,매도,매수,거래량,변동량


In [10]:
quote_tags = soup.find_all('tr', onmouseover="mouseOver(this)")

In [11]:
quote_tags[0]

<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">15:58</span></td>
<td class="num"><span class="tah p11">58,800</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				400
				</span>
</td>
<td class="num"><span class="tah p11">58,900</span></td>
<td class="num"><span class="tah p11">58,800</span></td>
<td class="num"><span class="tah p11">17,894,851</span></td>
<td class="num"><span class="tah p11">167</span></td>
</tr>

In [12]:

for raw in quote_tags:
    record = []
    #각 raw의 체결 시각 구하기
    time = raw.select_one('.tah.p10.gray03').text
    #각 raw의 체결가 구하기
    Multidata = raw.select('span.tah.p11')
    fill_price = Multidata[0].text.strip()
    sell_price = Multidata[1].text.strip()
    buy_price = Multidata[2].text.strip()
    trading_vol = Multidata[3].text.strip()
    volatility = Multidata[4].text.strip()
    # 각 raw의 전일비 구하기    
    ratio = raw.select_one('.tah.p11.red02').text.strip()
    record = [time, fill_price,ratio,sell_price,buy_price,trading_vol,volatility]
    data.loc[len(data)] = record

In [13]:
data

,체결시각,체결가,전일비,매도,매수,거래량,변동량
0,15:58,"58,800",400,400,"58,900","58,800","17,894,851"
1,15:57,"58,800",400,400,"58,900","58,800","17,894,684"
2,15:56,"58,800",400,400,"58,900","58,800","17,892,566"
3,15:55,"58,800",400,400,"58,900","58,800","17,892,544"
4,15:54,"58,800",400,400,"58,900","58,800","17,891,478"
5,15:53,"58,800",400,400,"58,900","58,800","17,890,915"
6,15:52,"58,800",400,400,"58,900","58,800","17,890,871"
7,15:51,"58,800",400,400,"58,900","58,800","17,890,867"
8,15:50,"58,800",400,400,"58,900","58,800","17,890,761"
9,15:49,"58,800",400,400,"58,900","58,800","17,890,739"


In [14]:
data.to_csv('stock_price.csv')